# Datenbankzugriff

Für die folgenden Beispiele wollen wir eine Tabelle Personen(nr, name) mit den Attributen `nr` und `Name` erstellen und jeweils wieder abfragen.

In [1]:
TESTDATEN = [[101, "Peter"], 
             [102, "Petra"], 
             [103, "Hans"], 
             [104,"Claudia"]]

## Sqlite

Mit dem Python-Paket [sqlite3](https://docs.python.org/library/sqlite3.html)
kann auf  sqlite-Datenbanken direkt zugegriffen werden. Es muss daher nicht
installiert werden und ist bei jeder Python-Installation vorhanden.

In der folgenden Beispielanwendung erstellen wir eine Beispieltabelle,
fügen ein paar Daten hinzu und geben diese wieder aus.

In [2]:
import sqlite3

Nach dem Import des `sqlite3`-Moduls können nun Daten hinzugefügt und anschließend wieder ausgelesen werden.
Zunächst wird eine Verbindung erstellt und aus dieser ein `Cursor` generiert, mit dem auf die Datenbank zugegriffen werden kann.

In [3]:
def insert_into_db():
    conn = sqlite3.connect("datenbank.db")
    c = conn.cursor()
    c.execute("""CREATE TABLE IF NOT EXISTS 
        personen(nr int, name text)""")

    for nr, name in TESTDATEN:
        # use ? to avoid sql injection
        print("Füge Daten hinzu:", nr, name)
        c.execute(
            """INSERT INTO personen (nr, name) VALUES(?,?)""", 
            (nr, name))

    conn.commit()
    conn.close()

insert_into_db()

Füge Daten hinzu: 101 Peter
Füge Daten hinzu: 102 Petra
Füge Daten hinzu: 103 Hans
Füge Daten hinzu: 104 Claudia


Wir finden nun eine Datei `datenbank.db` im aktuellen Verzeichnis.

In [4]:
! ls datenbank.db

datenbank.db


Die Daten können wieder aus der Datei gelesen werden.

In [5]:
def select_from_db():
    conn = sqlite3.connect("datenbank.db")
    c = conn.cursor()
    rows = c.execute("SELECT nr, name FROM personen")

    print("Nr.\t Name")
    for i, name in rows:
        print(i, "\t", name)

    conn.close()

select_from_db()


Nr.	 Name
101 	 Peter
102 	 Petra
103 	 Hans
104 	 Claudia


Zum Schluss werden alle Daten wieder gelöscht.

In [6]:
def del_data():
    conn = sqlite3.connect("datenbank.db")
    c = conn.cursor()
    c.execute("DELETE FROM personen")
    
    conn.commit()
    conn.close()
    
del_data()

Wir löschen auch die Datenbank-Datei.

In [7]:
! rm datenbank.db

## MySQL

Für den Zugriff auf eine MySQL-Datenbank muss zusätzlich ein 
[Connector für 
MySQL](https://dev.mysql.com/doc/connector-python/en/connector-python-installation-binary.html)
installiert werden. Dies kann z.B. mit dem folgenden Aufruf geschehen.

    $ pip install mysql-connector-python
    
Wenn alles geklappt hat, können wir im Anschluss das neue Modul importieren.

In [8]:
import mysql.connector

Wie stellen eine Verbindung mit der Datenbank her. Diesmal müssen wir Zugangsdaten und die Datenbank angeben.

In [9]:
cnx = mysql.connector.connect(user='root', password='',
                              host='127.0.0.1',
                              database='test')
cnx.close()

Nun können wir eine Tabelle `personen` in der Datenbank `test` erstellen.

In [10]:
conn = mysql.connector.connect(user='root', password='',
                              host='127.0.0.1',
                              database='test')
c = conn.cursor()
c.execute("""DROP TABLE IF EXISTS personen""")
c.execute("""CREATE TABLE IF NOT EXISTS 
          personen(nr int, name text)""")

cnx.close()

Wir benutzen das Kommandozeilentool `mysql` und lassen uns die Tabellen in der Datenbank `test` anzeigen.

In [11]:
! mysql -uroot --execute="SHOW TABLES;" test 

+----------------+
| Tables_in_test |
+----------------+
| personen       |
+----------------+


Nun können wir neue Daten in die Tabelle einfügen.

**Achtung:** Die Platzhalter beim Einfügen werden unter MySQL anders angegeben. Außerdem muss die Anweisung mit einem COMMIT in die Datenbank übertragen werden.

In [12]:
conn = mysql.connector.connect(user="root", password="",
                              host="127.0.0.1", database="test")
c = conn.cursor()
for nr, name in TESTDATEN:
    print("Füge Daten hinzu:", nr, name)
    c.execute(
        """INSERT INTO personen (nr, name) VALUES(%s,%s)""", 
        (nr, name))
conn.commit()
conn.close()

Füge Daten hinzu: 101 Peter
Füge Daten hinzu: 102 Petra
Füge Daten hinzu: 103 Hans
Füge Daten hinzu: 104 Claudia


Schließlich werden die Daten wieder mit einer SELECT-Anweisung aus der Datenbank abgefragt. Wir nutzen zunächst den Kommandozeilen-Client.

In [13]:
! mysql -uroot --execute="SELECT * FROM personen" test 

+------+---------+
| nr   | name    |
+------+---------+
|  101 | Peter   |
|  102 | Petra   |
|  103 | Hans    |
|  104 | Claudia |
+------+---------+


Die Daten können natürlich auch mit Python abgerufen werden. Auch hier gibt es einen Unterschied zu der Version mit sqlite: `execute` liefert auch für Abfragen keinen Rückgabewert. Stattdessen wird über den Cursor iteriert.

In [14]:
conn = mysql.connector.connect(user="root", password="",
                               host="127.0.0.1", database="test")
c = conn.cursor()
c.execute("SELECT nr,name FROM personen")
for nr, name in c:
    print(nr, name)
    
conn.close()

101 Peter
102 Petra
103 Hans
104 Claudia


Die weitere Dokumentation von MySQL enthält verschiedene [Beispiele, die
den Umgang mit einer MySQL-Datenbank 
zeigen](https://dev.mysql.com/doc/connector-python/en/connector-python-examples.html).
Der wesentliche Unterschied besteht darin, wie eine Verbindung aufgebaut wird.
Nach dem Erzeugen eines Cursors mit ``cursor = eine_connection.cursor()`` kann
mit einem Aufruf von ``cursor.execute(...)`` wie im Falle von sqlite auf die 
Datenbank zugegriffen werden.

Das sehr ausführliche [Python MySQL-Tutorial](https://pynative.com/python-mysql-tutorial/)
geht auf viele Details genauer ein.